In [126]:
import random
import numpy as np
import matplotlib.pyplot as plt

from genetics import GeneticRemovingZeros
import utils

import importlib as im
im.reload(utils)

<module 'utils' from '/Users/sergey/AIU/routes/utils.py'>

In [2]:
class Hungarian:
    rounding_value = 2

    def __init__(
        self,
        random_generate : bool = False,
        checkpoints_cnt : int = 10,
        minmax : tuple = (5, 15),
        data : list = [[],[]]):
        """инициализация объекта венгерского алгоритма

        Args:
            random_generate (bool, optional): генерация случайной карты. Defaults to False.
            checkpoints_cnt (int, optional): количество точек (используется только при генерации карты). Defaults to 10.
            minmax (tuple, optional): минимальное и максимальное значение времени между двумя точками (используется только при генерации карты). Defaults to (5, 15).
            data (list, optional): карта, заданная пользователем (используется при значении random_generate=False). Defaults to [[],[]].
        """
        if random_generate:
            self.start_field = np.random.randint(minmax[0], minmax[1], size=(checkpoints_cnt, checkpoints_cnt))
        else:
            self.start_field = np.array(data)

        self.max_time = self.start_field.max()+1
        np.fill_diagonal(self.start_field, self.max_time)
        self.checkpoints_cnt = self.start_field.shape[0]
        self.current_field = utils.rounding(
            field=self.start_field.copy(),
            rounding_value=self.__class__.rounding_value)


    def reduce(self) -> None:         
        """Выполнение операции вычитания минимальных значений по строкам и столбцам
        """
        self.current_field = (self.current_field.T - utils.get_min(self.current_field, True)).T
        self.current_field -= utils.get_min(self.current_field, False)
        np.fill_diagonal(self.current_field, self.max_time)


    def check(self) -> np.array:
        """Поиск возможных решений

        Returns:
            np.array: найденные решения
        """
        def recursive_search(checkpoint:int, path:list) -> None:
            """Рекурсивная проверка существования решения

            Args:
                checkpoint (int): номер точки
                path (list): маршрут
            """
            if len(path)==self.checkpoints_cnt:
                exists_path.append(path)
                return
            if checkpoint in path:
                return        
            path.append(checkpoint)        
            for value in checkpoit_times[checkpoint]:
                recursive_search(value, path.copy())                
            
        exists_path = []
        rows, columns = np.where(self.current_field==0)
        checkpoit_times = {i:[] for i in range(self.checkpoints_cnt)}
        for i, rr in enumerate(rows):
            checkpoit_times[rr].append(columns[i])
        for i in range(self.checkpoints_cnt):
            recursive_search(i, [])
        if len(exists_path)==0:
            return exists_path
        return np.unique(exists_path, axis=0)

In [143]:
hung = Hungarian(random_generate=True, checkpoints_cnt=8, minmax=(4, 27))
hung.reduce()
print(hung.current_field)
hung.check()


[[27  0 16  0  6 16 14 16]
 [ 0 27 18  0  6 12  2  6]
 [10  8 27 16  2  0  0  2]
 [ 0  4  6 27 16  4 20 10]
 [ 4  0 10  0 27 10 16  2]
 [12  0 16 18  8 27 14  0]
 [16  8  0  2  0  2 27  2]
 [22  0 16 10 12 12  4 27]]


[]

In [146]:
genetic = GeneticRemovingZeros(checkpoint_count=8)

In [151]:
cut_index = genetic.process(hung.current_field)
cut_index

array([                   0,                    0,                    1,
                          0,                    0,                    1,
                          1,                    0,                    1,
                          1,                    0, -7463636586828776592,
                          0,                    0,                    0,
                          0])

In [148]:
utils.cut_matrix(hung.current_field, cut_index)

array([[16,  6, 16, 14, 16],
       [18,  6, 12,  2,  6],
       [ 6, 16,  4, 20, 10],
       [10, 27, 10, 16,  2],
       [16, 12, 12,  4, 27]])

In [149]:
hung.current_field

array([[27,  0, 16,  0,  6, 16, 14, 16],
       [ 0, 27, 18,  0,  6, 12,  2,  6],
       [10,  8, 27, 16,  2,  0,  0,  2],
       [ 0,  4,  6, 27, 16,  4, 20, 10],
       [ 4,  0, 10,  0, 27, 10, 16,  2],
       [12,  0, 16, 18,  8, 27, 14,  0],
       [16,  8,  0,  2,  0,  2, 27,  2],
       [22,  0, 16, 10, 12, 12,  4, 27]])

In [150]:
cut_index

array([                  0,                   0,                   1,
                         0,                   0,                   1,
                         1,                   0,                   1,
                         1,                   0, -176516166584348187,
                         0,                   0,                   0,
                         0])